In [ ]:
def dist_all_to_all(self, grid_reordered):
    '''
    Method for calculating non-euclidian distances in GemPy. Four required steps are wrapped in separate methods.
    Args:
        grid_reordered: Spatial (x,y,z) coordinates of discretized GemPy grid, reordered in random path as
                        defined for SGS run. Also containing corresponding scalar field values.
    Returns:
        dist_matrix: Matrix containing distances from each discrete grid point to each other grid point, 
                    ordered as defined for SGS path.
    '''
    # 1: Calculate reference plane within domain between top and bottom border (based on scalar field value)
    med_ver, med_sim, grad_plane = self.create_central_plane()
    
    # 2: Projection of each point in domain on reference plane (by closest point) and save reference point
    #    Definition of perpendicular distance portion either by method A or method B
    ref, perp = self.projection_of_each_point(med_ver, grad_plane, grid_reordered)
    
    # 3: Calculate all distances between vertices on reference plane by heat method
    dist_clean = self.proj_surface_dist_each_to_each(med_ver, med_sim)
    
    # 4: Combine results to final distance matrix, applying anisotropy factor if desired
    dist_matrix = self.distances_grid(ref, perp, dist_clean, grid_reordered)

    return dist_matrix

In [ ]:
def create_central_plane(self):
    '''
    Method to create reference plane.
    Args:
        (from class) grid_dataframe.scalar: Scalar values of discretized grid points within selected domain
        (from GemPy) geomodel: Lithology block and inforamtion from GemPy model
        (from GemPy) extent: Extent of GemPy model
        (from GemPy) resolution: Resolution of GemPy model
    Returns:
        vertices: Vertices of triangular mesh of reference plane
        simplices: Simplices of triangular mesh of reference plane
        grad: scalar field value of reference plane
    '''
    # find highest and lowest scalar field value (top and bot layer) in domain
    grad_top = np.max(self.grid_dataframe.scalar)
    grad_bot = np.min(self.grid_dataframe.scalar)
    
    # reshape scalar field values in domain for marching cubes algorithm
    a = self.geomodel[1].reshape(self.resolution)
    
    # calculate central scalar field value
    grad = (grad_top + grad_bot) / 2

    # create reference plane with marching cubes
    vertices, simplices, normals, values = measure.marching_cubes_lewiner(
        a,
        grad,
        step_size=1,
        spacing=((self.extent[1] / self.resolution[0]), (self.extent[3] / self.resolution[1]),
                     (self.extent[5] / self.resolution[2])))
    
    return vertices, simplices, grad

In [ ]:
def projection_of_each_point(self, ver, plane_grad, grid_reordered:
    '''
    Method to create reference plane.
    Args:
        (from class) distance_type: Choose between method A (deformed_A) or method B (deformed_B)
        (from class) grid_dataframe.scalar: Scalar values of discretized grid points within selected domain
        grid_reordered: Spatial (x,y,z) coordinates of discretized GemPy grid, reordered in random path as
                        defined for SGS run. Also containing corresponding scalar field values.
        
    Returns:
        ref: array containing reference vertex on reference plane for each grid point
        perp: array containing perpendicular portion of distance for each grid point
    '''
                             
    # extract spatial coordinates
    grid = grid_reordered[:, :3]
    # create control arrays to check on which side of reference plane points are located
    grad_check = grid_reordered[:, 4] 
    grad_check = grad_check > plane_grad
    # create empty array for reference vertices and perpendicualr distances
    ref = np.zeros(len(grid))
    perp = np.zeros(len(grid))
    # precalculations required for method B                     
    grad_top = np.max(self.grid_dataframe.scalar)
    grad_bot = np.min(self.grid_dataframe.scalar)
    ave_thick = 200 # example value
    grad_dist = grad_top - grad_bot
    fact = ave_thick / grad_dist

    # loop through grid to reference each grid point to closest vertex on reference plane by index
    for i in range(len(grid)):
        # save reference to closest vertex on reference plane
        ref[i] = cdist(grid[i].reshape(1, 3), ver).argmin()
        if self.distance_type == 'deformed_A':
            # save direct distance to closest vertex on reference plane (method A)
            perp[i] = cdist(grid[i].reshape(1, 3), ver).min()
        elif self.distance_type == 'deformed_B':    
            # calculate perpendicular distance portion based on scalar field value
            perp[i] = (grad_check[i] - plane_grad) * fact

    # check on which side of reference plane grid point is located
    # and make distance either positive or negative based on that
    for i in range(len(perp)):
        if grad_check[i] == True:
            perp[i] = perp[i] * (-1)

    return ref, perp

In [ ]:
def proj_surface_dist_each_to_each(self, med_ver, med_sim):
    '''
    Method to calculate distance between all vertices on reference plane based on Heat Method.
    Args:
        med_ver: vertices of reference plane
        med_sim: simplices of reference plane
    Returns:
        dist_clean: symmetric matrix of distances between all vertices on reference plane
    '''
    # precomputing mesh for heat method (Crane et al., 2013; Neumann et al., 2013)
    compute_distance = GeodesicDistanceComputation(med_ver, med_sim)

    # create empty matrix
    dist = np.zeros([len(med_ver), len(med_ver)])

    for i in range(len(med_ver)):
        # distance calculation based on heat method (Crane et al., 2013; Neumann et al., 2013)
        dist[i] = compute_distance(i)

    # take average of distances as heat method is not exact - creating symmetric matrix
    dist_clean = (dist + dist.T) / 2

    return dist_clean

In [ ]:
def distances_grid(self, ref, perp, dist_clean, grid_reordered):
    '''
    Method to combine all calculated distances
    Args:
        ref: array containing reference vertex on reference plane for each grid point
        perp: array containing perpendicular portion of distance for each grid point
        dist_clean: symmetric matrix of distances between all vertices on reference plane
        grid_reordered: Spatial (x,y,z) coordinates of discretized GemPy grid, reordered in random path as
                        defined for SGS run. Also containing corresponding scalar field values.
    Returns:
        dist_matrix: final matrix containing non-euclidian distances between all grid points
    '''
    # create empty matrix, convert references to int
    dist_matrix = np.zeros([len(ref), len(ref)])
    ref = ref.astype(int)

    # Loop through matrix entries
    #for i in range(len(ref)):
    #    for j in range(len(ref)):
    #        # set distance by pythagorean theorem from perpendicular and parallel portion
    #        # an_factor is included here, set to one (no anisotropy) by default
    #        dist_matrix[i][j] = np.sqrt(((dist_clean[ref[i]][ref[j]] / self.an_factor))**2+(abs(perp[i] - perp[j])**2))
    
    
    ref_range = np.arange(ref.shape[0])
    i, j = np.meshgrid(ref_range, ref_range)
    dist_matrix[i][j] = np.sqrt(((dist_clean[ref[i]][ref[j]] / self.an_factor))**2+(abs(perp[i] - perp[j])**2))
    
    # secure that diagonal is zero
    np.fill_diagonal(dist_matrix, 0)

    return dist_matrix